# Data Collection 2: DrugBank.com

##### Disclaimer
Use of DrugBank.com data is for research purposes only.

## Overview
This notebook includes codes used to scrape drug information on the [DrugBank.com](https://www.drugbank.com/) website.

In order to create my own dataset, I gathered more information on DrugBank.com, by searching each drug from the Yellow Card reports.

Again, I first imported the nexessary libraries.

In [1]:
# import necessary pacakges
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
from time import sleep
import random
import pandas as pd
from tqdm import tqdm
import string

## Obtain list of drugs
I first load the `yellow_card_links.csv` file created from the previous notebook `1a_yellow_card` to obtain the list of drugs in the Yellow Card reports.

I then create a new dataframe, with just the drug names and Yellow Card IDs.

In [41]:
# import yellow card links csv to get drug name list
yc = pd.read_csv('yellow_card_links.csv', dtype = object)

# create dataframe with an empty drug column
drugbank = pd.DataFrame(columns = ['drug'])
# convert drug names to lowercase and remove [] brackets
drugbank['drug'] = yc['drug_name'].str.lower().str.replace('[','').str.replace(']','')
# copy over the yc_id column
drugbank['yc_id'] = yc['yc_id']
drugbank.head()

,drug,yc_id
0,abacavir,000040046536
1,abatacept,000561378321
2,abciximab,000231911819
3,abemaciclib,000369408139
4,abiraterone,000968368347


## Use Selenium to scrape DrugBank.com
### Set Up Selenium
Let's first set up the necessary settings for Selenium Web Driver:

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# set up webdriver for Selenium
options = webdriver.ChromeOptions()
chrome_options=options
options.add_argument('--enable-javascript')
driver = webdriver.Chrome(executable_path='/Applications/chromedriver', options=options)

# go to drugbank website
driver.get('https://go.drugbank.com/drugs/DB00316')

### Obtain DrugBank IDs
Now let's start scraping. This was done in smaller chunks of 500 drugs to prevent errors. First, I obtain the unique DrugBank IDs for each drug.

In [30]:
# get list of drugs to loop through
drugs = drugbank['drug']

# set up an empty list to save scraped information
db_ids = []

# loop through each drug
for drug in tqdm(drugs):
    
    try:
        # click on search box
        driver.find_element_by_xpath('/html/body/header/nav[2]/div[1]/form/div[2]').click()
        # enter search keyword
        searchbox = driver.find_element_by_xpath('//*[@id="query"]')
        # clear search box
        searchbox.clear()
        # enter search word
        searchbox.send_keys(drug)
        # click enter
        searchbox.send_keys(Keys.ENTER)
        
        # wait 5 seconds
        sleep(5)
        
        try:
            # method 1 - get DrugBank number using xpath
            num = driver.find_element_by_xpath('/html/body/main/div/div/div[2]/div[2]/dl[1]/dd[2]').text
            db_ids.append(num)
        
        except:
            # method 2 - get DrugBank number using class
            num = driver.find_element_by_class_name("col-xl-4 col-md-9 col-sm-8").text
            db_ids.append(num)
            
        # sleep for a few seconds
        sleep(random.randint(5, 8))
    
    
    except:
        # if not successfully scraped, add a null value
        db_ids.append(np.nan)

100%|██████████| 421/421 [1:21:27<00:00, 11.61s/it]


In [42]:
# check length of list
len(db_ids)

2339

In [45]:
# add to drugbank data frame
drugbank['db_id'] = db_ids
# show first 5 rows
drugbank.head()

,drug,yc_id,db_id
0,abacavir,000040046536,DB01048
1,abatacept,000561378321,DB01281
2,abciximab,000231911819,DB00054
3,abemaciclib,000369408139,DB12001
4,abiraterone,000968368347,DB05812


In [46]:
# brief overview
drugbank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   drug    2339 non-null   object
 1   yc_id   2339 non-null   object
 2   db_id   1981 non-null   object
dtypes: object(3)
memory usage: 54.9+ KB


"Drugs" that do not have `db_id` are reports that involve plant extracts, experimental compounds, and other products that do not have any clinical data. As I am only focusing on medicinal drugs, these are excluded from the capstone.

### Obtain information on drug targets, pathways, and categories
Next, we use `pd.read_html` to get all the tables on each drug's page.
Since each drug has a different number of tables, I looped through each dataframe and checked for the column names.
Due to time constraint, only the drug targets (`targets`), pathways (`pathways`), and drug categories (`drug_cat`) are scraped.

> Note: `pathways` and `drug_cat` were not used in the end as less than a third of the drugs had the information available.

In [43]:
# set up headers for Selenium web driver
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"
accept = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
encoding = "gzip, deflate, br"
lang = "en-GB,en;q=0.9"
headers = {'accept': accept, 'accept-encoding': encoding,
           'accept-language': lang, 'user-agent': user_agent}

In [100]:
# create empty lists to store scraped information
targets = []
pathways = []
drug_cats = []

# loop throguh each DrugBank ID scraped earlier above
for db_id in tqdm(drugbank['db_id']):

    # add the drugbank id to the url to access each drug's page
    url = f'https://go.drugbank.com/drugs/{db_id}'
    r = requests.get(url, headers=headers)
    
    # get ahfs code (drug category)
    try:
        soup = BeautifulSoup(r.text, 'html.parser')
        results = soup.find_all("ul", class_ = "list-unstyled table-list")
        drug_cats.append(results[3].text)
    except:
        drug_cats.append(np.nan)
      
    
    # read all the tables on the webpage
    try:    
        # get dataframes
        dfs = pd.read_html(r.text)
    
    # error message that there was no table for the specific url
    except:
        print('no tables')
        targets.append(np.nan)
        pathways.append(np.nan)
    
    else:
        target = 0
        pathway = 0
        
        # loop through each dataframe stored in dfs
        for df in dfs:
            
            # extract targets
            if df.columns[0] == 'Target':
                target = 1
                targets.append([x[1:] for x in df['Target'].values])

            # extract pathways
            elif df.columns[0] == 'Pathway':
                pathway = 1
                pathways.append(list(df['Pathway'].values))

            else:
                pass
            
        # once all the dfs are looped through for each drug,
        # if no target table is found, append nan value
        if target == 0:
            targets.append(np.nan)
            
        # same for pathway table
        if pathway == 0:
            pathways.append(np.nan)
    
        # sleep
        sleep(random.randint(3, 6))

  1%|          | 28/2340 [02:27<2:39:18,  4.13s/it]

no tables


  1%|▏         | 33/2340 [02:52<2:37:51,  4.11s/it]

no tables


  1%|▏         | 34/2340 [02:53<1:55:26,  3.00s/it]

...

 99%|█████████▉| 2325/2340 [2:48:18<00:32,  2.14s/it]

no tables


100%|█████████▉| 2330/2340 [2:48:40<00:32,  3.30s/it]

no tables


100%|██████████| 2340/2340 [2:49:31<00:00,  4.35s/it]


In [104]:
# check lengths of lists storing scraped information
len(targets), len(pathways), len(drug_cats)

(2340, 2340, 2340)

## Create DrugBank dataframe
Now we create a dataframe using the information scraped from the website, using the codes above.

In [106]:
# create dataframe from scraped info
drugbank['target'] = targets
drugbank['pathways'] = pathways
drugbank['drug_cat'] = drug_cats
drugbank['db_id'] = db_ids
drugbank.head()

,drug,yc_id,db_id,target,pathways,drug_cat
0,abacavir,000040046536,DB01048,"[Reverse transcriptase/RNaseH, HLA class I his...",[Abacavir Action Pathway],08:18.08.20 — Nucleoside and Nucleotide Revers...
1,abatacept,000561378321,DB01281,"[T-lymphocyte activation antigen CD80, T-lymph...",NaN,Bristol-Myers Squibb Co.\nCelltrion Inc.\nE.R....
2,abciximab,000231911819,DB00054,"[Integrin beta-3, Integrin alpha-IIb, Low affi...",[Abciximab Action Pathway],92:00.00 — Miscellaneous Therapeutic Agents
3,abemaciclib,000369408139,DB12001,"[Cyclin-dependent kinase 4, Cyclin-dependent k...",NaN,10:00.00 — Antineoplastic Agents
4,abiraterone,000968368347,DB05812,"[Steroid 17-alpha-hydroxylase/17,20 lyase]",NaN,10:00.00 — Antineoplastic Agents


In [107]:
drugbank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   drug      2340 non-null   object
 1   yc_id     2340 non-null   object
 2   db_id     1987 non-null   object
 3   target    1565 non-null   object
 4   pathways  402 non-null    object
 5   drug_cat  1072 non-null   object
dtypes: object(6)
memory usage: 109.8+ KB


## Data Cleaning

### Initial Overview
Now we clean the dataframe that we have just created.

In [108]:
# check for null values
drugbank.isnull().sum()

drug           0
yc_id          0
db_id        353
target       775
pathways    1938
drug_cat    1268
dtype: int64

In [109]:
# check shape of dataframe
drugbank.shape

(2340, 6)

### Manually fill in gaps
Since there was not a lot of drugs in the `not_found` list, I manually searched for the DrugBank IDs in case my Selenium codes did not find it in the first attempt.

In [110]:
# list of drugs not found while scraping
not_found = list(drugbank[drugbank['db_id'].isnull()]['drug'].values)
print(len(not_found))
# show first 20 values
not_found[:20]

353


['adenophora',
 'agrimonia',
 'agropyron repens',
 'alathon polyethylene resin',
 'alchemilla vulgaris',
 'alexitol',
 'allium sativum',
 'alpha-1 proteinase inhibitor',
 'alphadolone acetate',
 'alphaxalone',
 'althaea',
 'ambucetamide',
 'ambutonium',
 'aminoxidase',
 'amiphenazole',
 'anamirta cocculus',
 'anethum graveolens',
 'anti-d (rho) immunoglobulin',
 'anti-inhibitor coagulant complex, dried',
 'antilymphocyte immunoglobulins']

In [89]:
# for instance, I manually added the DB ids
x = drugbank[drugbank['drug'] =='sulphasalazine'].index
drugbank.loc[x, 'db_id'] = 'DB00795'
x = drugbank[drugbank['drug'] =='sulphamethoxypyridazine'].index
drugbank.loc[x, 'db_id'] = 'DB13773'
x = drugbank[drugbank['drug'] =='sulphamethoxydiazine'].index
drugbank.loc[x, 'db_id'] = 'DB00576'
x = drugbank[drugbank['drug'] =='sulphamethizole'].index
drugbank.loc[x, 'db_id'] = 'DB12829'
x = drugbank[drugbank['drug'] =='sulphadimidine'].index
drugbank.loc[x, 'db_id'] = 'DB01582'
x = drugbank[drugbank['drug'] =='sodium thiosulphate'].index
drugbank.loc[x, 'db_id'] = 'DB09499'
x = drugbank[drugbank['drug'] =='sodium picosulphate'].index
drugbank.loc[x, 'db_id'] = 'DB09268'
x = drugbank[drugbank['drug'] =='senna'].index
drugbank.loc[x, 'db_id'] = 'DB11365'
x = drugbank[drugbank['drug'] =='thiacetazone'].index
drugbank.loc[x, 'db_id'] = 'DB12829'
x = drugbank[drugbank['drug'] =='sacubitril/valsartan'].index
drugbank.loc[x, 'db_id'] = 'DB09292'
x = drugbank[drugbank['drug'] =='polymyxin'].index
drugbank.loc[x, 'db_id'] = 'DB00781'
x = drugbank[drugbank['drug'] =='lithium'].index
drugbank.loc[x, 'db_id'] = 'DB14507'

Here I looked at drugbank Ids that occured more than once, and amended the errors.

In [91]:
# check for drugbank ids that appeared more than once
dup_ids = list(drugbank.db_id.value_counts()[drugbank.db_id.value_counts() > 1].index)
dup_ids

['DB00066',
 'DB01053',
 'DB00052',
 'DB09449',
 'DB00067',
 'DB00863',
 'DB04398',
 'DB00783',
 'DB00085',
 'DB12829',
 'DB12667',
 'DB09146']

In [101]:
# show first 10
drugbank[drugbank.db_id.isin(dup_ids)].sort_values('db_id').head(10)

,drug,yc_id,db_id,target,pathways,drug_cat
2001,somatropin,000314288027,DB00052,"[Growth hormone receptor, Prolactin receptor]",NaN,Genentech inc\nSerono laboratories inc\nCangen...
988,growth hormone,000996119734,DB00052,"[Growth hormone receptor, Prolactin receptor]",NaN,Genentech inc\nSerono laboratories inc\nCangen...
900,follitropin delta,000058677892,DB00066,[Follicle-stimulating hormone receptor],NaN,Organon usa inc\nEmd serono inc
899,follitropin beta,000127709748,DB00066,[Follicle-stimulating hormone receptor],NaN,Organon usa inc\nEmd serono inc
897,follicle stimulating hormone,000322144909,DB00066,[Follicle-stimulating hormone receptor],NaN,Organon usa inc\nEmd serono inc
898,follitropin alfa,001040768484,DB00066,[Follicle-stimulating hormone receptor],NaN,Organon usa inc\nEmd serono inc
153,argipressin,000538552928,DB00067,"[Vasopressin V2 receptor, Vasopressin V1a rece...",NaN,Parke davis div warner lambert co
2270,vasopressin,000121577179,DB00067,"[Vasopressin V2 receptor, Vasopressin V1a rece...",NaN,Parke davis div warner lambert co
1593,pancrelipase,001001595052,DB00085,"[Dietary fat, Dietary protein, Dietary starch]",NaN,56:16.00 — Digestants
1592,pancreatin,000240328336,DB00085,"[Dietary fat, Dietary protein, Dietary starch]",NaN,56:16.00 — Digestants


In [78]:
# more manual corrections
drugbank.loc[379, 'db_id'] = 'DB01416'
drugbank.loc[371, 'db_id'] = 'DB01140'
drugbank.loc[372, 'db_id'] = 'DB00567'
drugbank.loc[373, 'db_id'] = 'DB00535'
drugbank.loc[374, 'db_id'] = 'DB01413'
drugbank.loc[369, 'db_id'] = np.nan
drugbank.loc[376, 'db_id'] = 'DB00493'
drugbank.loc[383, 'db_id'] = 'DB01415'
drugbank.loc[375, 'db_id'] = 'DB00671'
drugbank.loc[382, 'db_id'] = 'DB00438'
drugbank.loc[381, 'db_id'] = 'DB06590'
drugbank.loc[380, 'db_id'] = 'DB13499'
drugbank.loc[389, 'db_id'] = 'DB00482'
drugbank.loc[378, 'db_id'] = 'DB13682'
drugbank.loc[377, 'db_id'] = 'DB01331'
drugbank.loc[364, 'db_id'] = 'DB10516'
drugbank.loc[388, 'db_id'] = 'DB01112'
drugbank.loc[390, 'db_id'] = 'DB04846'
drugbank.loc[391, 'db_id'] = np.nan
drugbank.loc[392, 'db_id'] = 'DB14707'
drugbank.loc[363, 'db_id'] = 'DB01136'
drugbank.loc[394, 'db_id'] = 'DB09008'
drugbank.loc[395, 'db_id'] = 'DB00456'
drugbank.loc[396, 'db_id'] = 'DB01326'
drugbank.loc[398, 'db_id'] = 'DB09063'
drugbank.loc[387, 'db_id'] = 'DB01212'
drugbank.loc[399, 'db_id'] = 'DB00439'
drugbank.loc[400, 'db_id'] = 'DB13173'
drugbank.loc[401, 'db_id'] = 'DB08904'
drugbank.loc[362, 'db_id'] = np.nan
drugbank.loc[361, 'db_id'] = np.nan
drugbank.loc[360, 'db_id'] = np.nan
drugbank.loc[359, 'db_id'] = 'DB00521'
drugbank.loc[397, 'db_id'] = 'DB01333'
drugbank.loc[386, 'db_id'] = 'DB09050'
drugbank.loc[393, 'db_id'] = np.nan
drugbank.loc[384, 'db_id'] = 'DB01332'
drugbank.loc[365, 'db_id'] = 'DB00520'
drugbank.loc[366, 'db_id'] = np.nan
drugbank.loc[367, 'db_id'] = 'DB14298'
drugbank.loc[368, 'db_id'] = np.nan
drugbank.loc[385, 'db_id'] = 'DB04918'
drugbank.loc[370, 'db_id'] = 'DB00833'

In [95]:
# amend trometamol
drugbank.loc[2231]['db_id'] = 'DB03754'
drugbank.loc[2231]['target'] = ['Amyloid beta A4 protein']
drugbank.loc[2231]['pathways'] = np.nan
drugbank.loc[2231]['drug_cat'] = '40:08.00 — Alkalinizing Agents'

# amend gallium
drugbank.loc[934]['db_id'] = 'DB15494'
drugbank.loc[934]['target'] = ['Somatostatin receptor type 2', 'Somatostatin receptor type 5',
                               'Somatostatin receptor type 3', 'Somatostatin receptor type 1']

I also noticed that information on iodine was missing and manually added that in.

In [98]:
# check if iodine is in the dataframe
drugbank[drugbank.drug.str.contains('iodine')]

,drug,yc_id,db_id,target,pathways,drug_cat
307,cadexomer-iodine,000743544955,NaN,NaN,NaN,NaN
1114,iodobenzylguanidine m- (iodine-131),000501603764,NaN,NaN,NaN,NaN
1765,povidone-iodine,000850028924,DB06812,NaN,NaN,NaN


In [99]:
# add in information for iodine

# create variable t for iodine's targets
t = ['Arachidonate 5-lipoxygenase',
'Prostaglandin G/H synthase 2',
'Prostaglandin G/H synthase 1',
'Peroxisome proliferator-activated receptor gamma',
'Inhibitor of nuclear factor kappa-B kinase subunit',
'Inhibitor of nuclear factor kappa-B kinase subunit',
'Cystine/glutamate transporter',
'Acetyl-CoA acetyltransferase, mitochondrial',
'Thromboxane-A synthase',
'Phospholipase A2']

# append iodine row and save to dataframe
drugbank = drugbank.append(
    {'drug': 'iodine',
     'yc_id': '000743544955',
     'target': ['Sodium/iodide cotransporter', 'Microbial proteins'],
     'drug_cat': ['92:01.00 — Herbs and Natural Products', '88:29.00 — Minerals', '84:04.92 — Miscellaneous Local Anti-infectives'],
     'db_id': 'DB05382'},
    ignore_index = True)

After making the manual corrections above, I re-ran the web-scrape code in part 1.3.3, and moved onto the next step.

### Data cleaning
#### Pathways column
I dropped the pathways column as over 80% of the drugs did not have such information.

In [111]:
# drop pathways as not enough data
drugbank.drop(columns = ['pathways'], inplace = True)

#### Drug target column
I filled null values with a string of `'nan'`, so that it will be a category on its own when I dummify this column at a later stage.

In [118]:
drugbank['target'][drugbank['target'].isnull()] = 'nan'

#### Drug category column
I filled null values with a string of `'nan'`, so that it will be a category on its own when I dummify this column at a later stage.

In [112]:
drugbank['drug_cat'][drugbank['drug_cat'].isnull()] = 'nan'

This column also needs some cleaning, as the list of categories are saved as one long string with to delimiter `'\n'` included, for instance:

In [113]:
drugbank.loc[22, 'drug_cat']

'84:04.06 — Antivirals\n08:18.32 — Nucleosides and Nucleotides'

In [114]:
# split on \n
drugbank['drug_cat'] = drugbank['drug_cat'].apply(lambda x: x.split('\n'))

#### Drop rows without db_id
I then decided to only retain rows that have the drugbank IDs. Now I have 2326 rows, compared to 2339 initially.

In [120]:
drugbank = drugbank[~drugbank['db_id'].isnull()].copy()
drugbank.reset_index(drop = True, inplace = True)
drugbank.shape

(1987, 5)

In [123]:
drugbank.head()

,drug,yc_id,db_id,target,drug_cat
0,abacavir,000040046536,DB01048,"[Reverse transcriptase/RNaseH, HLA class I his...",[08:18.08.20 — Nucleoside and Nucleotide Rever...
1,abatacept,000561378321,DB01281,"[T-lymphocyte activation antigen CD80, T-lymph...","[Bristol-Myers Squibb Co., Celltrion Inc., E.R..."
2,abciximab,000231911819,DB00054,"[Integrin beta-3, Integrin alpha-IIb, Low affi...",[92:00.00 — Miscellaneous Therapeutic Agents]
3,abemaciclib,000369408139,DB12001,"[Cyclin-dependent kinase 4, Cyclin-dependent k...",[10:00.00 — Antineoplastic Agents]
4,abiraterone,000968368347,DB05812,"[Steroid 17-alpha-hydroxylase/17,20 lyase]",[10:00.00 — Antineoplastic Agents]


In [121]:
drugbank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   drug      1987 non-null   object
 1   yc_id     1987 non-null   object
 2   db_id     1987 non-null   object
 3   target    1987 non-null   object
 4   drug_cat  1987 non-null   object
dtypes: object(5)
memory usage: 77.7+ KB


In [122]:
## save dataframe
# drugbank.to_csv('drugbank_final.csv', index = False, header = True)

# Next step:
Now let's move onto the last notebook on data collection: `1c_reviews`